# ReThink Media Twitter API

This notebook is for the development and exploration of code for ReThink Media's Twitter API Python interface. The main goals of this notebook are:

- Search Tweets: query, date (optional)
  - Past seven days
  - Past 30 days
  - Full archive
  - Language = English
- Collect Tweets in .csv file
- Add data visualization
  - Top hashtags, keywords, influencers
  - Volume over time for queries/topics

In [11]:
# importing necessary modules
from dotenv import load_dotenv
import os
import json
import numpy as np
import pandas as pd
import tweepy

load_dotenv()

True

## Authentication

The variables below are what allow access to the Twitter API. I've defined them in a `.env` file, and I'm retrieving them with the code below. We then pass those variables in to a tweepy client in order to instantiate a Twitter API instance.

In [12]:
# retrieving environment variables
consumer_key = os.getenv("API_KEY")
consumer_secret = os.getenv("API_KEY_SECRET")
bearer_token = os.getenv("BEARER_TOKEN")
access_token = os.getenv("ACCESS_TOKEN")
access_secret = os.getenv("ACCESS_SECRET")

In [13]:
# Twitter API authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [14]:
# function to initialize Twitter API v1.1 instance (for 30-day and full archive search)
def init_api_1():
    
    # importing necessary modules and loading .env file
    from dotenv import load_dotenv
    import os
    import tweepy
    load_dotenv()
    
    # retrieving environment variables from .env file
    consumer_key = os.getenv("API_KEY")
    consumer_secret = os.getenv("API_KEY_SECRET")
    bearer_token = os.getenv("BEARER_TOKEN")
    access_token = os.getenv("ACCESS_TOKEN")
    access_secret = os.getenv("ACCESS_SECRET")
    
    # Twitter API authentication
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    # instantiating Twitter API v1.1 reference
    api_1 = tweepy.API(auth)
    
    return api_1

In [15]:
# function to initialize Twitter API v2 instance (for 7-day search)
def init_api_2():
    # importing necessary modules and loading .env file
    from dotenv import load_dotenv
    import os
    import tweepy
    load_dotenv()
    
    # retrieving environment variables from .env file
    consumer_key = os.getenv("API_KEY")
    consumer_secret = os.getenv("API_KEY_SECRET")
    bearer_token = os.getenv("BEARER_TOKEN")
    access_token = os.getenv("ACCESS_TOKEN")
    access_secret = os.getenv("ACCESS_SECRET")
    
    # instantiating Twitter API v2 reference
    api_2 = tweepy.Client(bearer_token=bearer_token,
                         consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         access_token=access_token,
                         access_token_secret=access_secret)
    
    return api_2

## Recent Search

The search function available to us in the Standard API package restricts our search to the past seven days, without a premium API dev subscription. For searches further back in the archive, we need to subscribe to a premium API dev environment or upgrade to the Academic API package, which is given to researchers with a clear thesis or research paper goal in mind.

The query can be 512 characters maximum, and the user can specify a `start_time` and `end_time` (as `datetime` or `str` objects) within the past seven days. The user can also search for hashtags as well. The default behavior for white space is "AND" joins, e.g., hello world = hello AND world. More information about Twitter API queries can be found [in their documentation](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query).

The 7-day search can receive an unlimited number of requests and 500,000 Tweets per month.

In [110]:
# instantiating a Twitter API v2 instance
api2 = tweepy.Client(bearer_token=bearer_token,
                consumer_key=consumer_key,
                consumer_secret=consumer_secret,
                access_token=access_token,
                access_token_secret=access_secret)
api2

In [111]:
# searching for "hello world" over the past seven days.
test_field = "text"
response = api2.search_recent_tweets(query="hello world lang:en", max_results=20, tweet_fields=test_field)

The `response` object is a tuple, and it consists of four items: `(data, includes, errors, meta)`.

The `data` object contains the Tweets that are retrieved, and `meta` is the metadata for those Tweets. In this reponse object, `includes` and `errors` are empty, so I'm not sure what `includes` is yet.

In [112]:
# printing Tweets
for i in range(len(response[0])):
    print(f"Tweet {i}:")
    print(response[0][i][test_field])

Tweet 0:
Hello World! Here's an interesting post! https://t.co/xPFYuCBtak
Tweet 1:
RT @polaristhelight: Hello boys and girls, welcome to Circus Baby's Pizza World 🎂🎉🍕
#blacktober #FNAF https://t.co/IWgcDgsnak
Tweet 2:
RT @TheMadTrooper: @Coinbase_NFT Hello! @Coinbase_NFT,
I am a #MadTrooper!
Welcome to the world of #NFTs
Tweet 3:
RT @phoyaTweets: Hello World
Help me win That Cole piece.... 
simply Retweet
Thank you in advance 🔥
@taylormade_cust
#Taylormadecustomsmw
Tweet 4:
RT @TheMadTrooper: @Coinbase_NFT Hello! @Coinbase_NFT,
I am a #MadTrooper!
Welcome to the world of #NFTs
Tweet 5:
Hello, world! (2021-10-21 18:26:29.116051)
Tweet 6:
shut up i'm literally saying hello to his hope world
Tweet 7:
Hello #NFT world! I bring you the Super Rare Old Photos from the 1850s-1950s. Each one is unique.

Let's start with photo from April 9,1953. Civil Defense air raid drill from a school. What a Life!

https://t.co/y2jI7xjowY

#NFT #NFTs #nftcollector #NFTcollection #NFTCommmunity
Tweet 8:
"Hell

In [94]:
# printing metadata for Tweets in response
response[3]

{'newest_id': '1450885307889426436',
 'oldest_id': '1450884517594546179',
 'result_count': 20,
 'next_token': 'b26v89c19zqg8o3fpdv5sr0kag6odo2pxlfk0m3q1pmyl'}

In [54]:
# retrieving full text of retweeted Tweet
retweet_id = response[0][0].referenced_tweets[0].id
retweet = api2.get_tweet(retweet_id)
retweet[0].text

"Hello world! 👶\n\nI wanted to wait until Twitter updates my Twitter card but they're taking too long 🤷\u200d♀️\n\nI am here to help you build 💪 &amp; launch 🚀 your #NoCode projects using at least 1 of the participating NoCode tools!\n\nClaim your free tickets NOW! https://t.co/Ul652SvkhN"

In [65]:
response[0][0]['author_id']

In [105]:
# function to retrieve Tweets from the past 7 days relevant to a query
def search_7(query, start_date=None, end_date=None, max_results=20):
    
    # initializing API v1.1 instance
    api_2 = init_api_2()
    
    # parsing dates passed into function
    from dateutil import parser
    if start_date:
        start_date = parser.parse(start_date)
    if end_date:
        end_date = parser.parse(end_date)
    
    # retrieving Tweets between start_date and end_date relevant to query
    tweet_fields = ["text", "attachments", "author_id", "context_annotations", "conversation_id", "created_at",
                   "entities", "geo", "in_reply_to_user_id", "lang", "public_metrics", "referenced_tweets"]
    response = api_2.search_recent_tweets(query=f"({query}) lang:en",
                                         start_time=start_date,
                                         end_time=end_date,
                                         max_results=max_results,
                                         tweet_fields=tweet_fields)
    
    # adding Tweet data to DataFrame
    import pandas as pd
    
    tweets = pd.DataFrame(columns=tweet_fields)
    tweets.index.name = "Tweet ID"
    for i in range(len(response[0])):
        tweet = response[0][i]
        tweet_id = tweet.id
        tweet_data = {}
        for field in tweet_fields:
            if tweet[field]:
                tweet_data[field] = tweet[field]
            else:
                tweet_data[field] = None
        tweets.loc[tweet_id] = tweet_data
    return tweets

In [160]:
test = search_7("hello world", max_results=100)
print(len(test))
test

100


,text,attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,in_reply_to_user_id,lang,public_metrics,referenced_tweets
Tweet ID,,,,,,,,,,,,
1451278092152999940,RT @Franz_Ferdinand: Hello world. Paul is leav...,None,596458837,"[{'domain': {'id': '10', 'name': 'Person', 'de...",1451278092152999940,2021-10-21 20:03:55+00:00,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",None,None,en,"{'retweet_count': 228, 'reply_count': 0, 'like...","[(type, id)]"
1451278072716701714,@ManUtdMEN https://t.co/moBesziC47. Hello eve...,None,1376059345805447170,"[{'domain': {'id': '3', 'name': 'TV Shows', 'd...",1451243132885118984,2021-10-21 20:03:50+00:00,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",None,57299064,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","[(type, id)]"
1451278046166671367,@tito_mboweni @LuckyStarSA https://t.co/moBesz...,None,1376059345805447170,"[{'domain': {'id': '3', 'name': 'TV Shows', 'd...",1451260492970926080,2021-10-21 20:03:44+00:00,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",None,855262092,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","[(type, id)]"
1451278023941103628,@ESPNFC https://t.co/moBesziC47. Hello everyo...,None,1376059345805447170,"[{'domain': {'id': '3', 'name': 'TV Shows', 'd...",1451256150578577417,2021-10-21 20:03:39+00:00,"{'mentions': [{'start': 0, 'end': 7, 'username...",None,18091004,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","[(type, id)]"
1451277998989131783,@ManUtdMEN https://t.co/moBesziC47. Hello eve...,None,1376059345805447170,"[{'domain': {'id': '3', 'name': 'TV Shows', 'd...",1451273834833797122,2021-10-21 20:03:33+00:00,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",None,57299064,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","[(type, id)]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1451275307441287171,"RT @HeenaParnani: 500days back, Disha and her ...",None,943010924633321472,None,1451275307441287171,2021-10-21 19:52:51+00:00,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",None,None,en,"{'retweet_count': 48, 'reply_count': 0, 'like_...","[(type, id)]"
1451275304714989603,RT @mzbooksbot: Jonah took Mo… his daughter… a...,None,946506851125882881,None,1451275304714989603,2021-10-21 19:52:50+00:00,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",None,None,en,"{'retweet_count': 2, 'reply_count': 0, 'like_c...","[(type, id)]"
1451275292899782657,RT @robleamusic: Hello World! Listen to this s...,None,21507000,None,1451275292899782657,2021-10-21 19:52:47+00:00,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",None,None,en,"{'retweet_count': 1, 'reply_count': 0, 'like_c...","[(type, id)]"


In [90]:
test.iloc[0]["referenced_tweets"][0]

<ReferencedTweet id=1451247639517872132 type=retweeted

## 30-Day/Full Archive Search

We can access 30-day and full archive searches without an Academic API package with a premium development environment through the Twitter API. This requires interfacing with the API v1.1, as opposed to v2 in the Recent Search.

The 30-day search can receive 250 requests and 25,000 Tweets per month, while the full archive search can receive 50 requests and 5,000 Tweets per month.

In [9]:
# initializing API v1.1
api1 = tweepy.API(auth)

In [10]:
response_30 = api1.search_30_day(label="30day", query="hello world lang:en", maxResults=20)

In [11]:
len(response_30)

20

In [12]:
type(response_30[0])

tweepy.models.Status

The `tweepy.models.Status` object contains a lot of data about the Tweet, such as its text, its author, and various aspects of metadata about the Tweet's creation and interactions.

In [13]:
response_30[0]._json

{'created_at': 'Wed Oct 20 17:06:26 +0000 2021',
 'id': 1450871038364037120,
 'id_str': '1450871038364037120',
 'text': "RT @SaitamaDude: Hello #SaitamaWolfPack!!! \n\nI've made a youtube channel with a quick and easy video on how to purchase #Saitama easily bef…",
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1397553132,
  'id_str': '1397553132',
  'name': 'koko kaye',
  'screen_name': 'koko_kaye',
  'location': None,
  'url': None,
  'description': '“You are never too old to set another goal, or to dream a new dream.” C.S. Lewis\n\nMy tweets/retweets are not financial advice. \nDYOR | Join the #SaitamaWolfPack',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 870,
  'friends_count': 1046,
  'listed_co

In [151]:
for i in response_30[0]._json:
    print(i)

created_at
id
id_str
text
source
truncated
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
user
geo
coordinates
place
contributors
retweeted_status
is_quote_status
quote_count
reply_count
retweet_count
favorite_count
entities
favorited
retweeted
filter_level
lang
matching_rules


In [153]:
def search_30(query, start_date=None, end_date=None, max_results=20):
    # initializing API v1.1 instance
    api_1 = init_api_1()
    
    # parsing dates passed into function
    from dateutil import parser
    if start_date:
        start_date = parser.parse(start_date)
    if end_date:
        end_date = parser.parse(end_date)
    
    # retrieving Tweets from the past 30 days relevant to query
    response = api_1.search_30_day(label="30day",
                                  query=f"{query} lang:en",
                                  fromDate=start_date,
                                  toDate=end_date,
                                  maxResults=max_results)
    
    # creating DataFrame of Tweets
    import pandas as pd
    
    tweet_fields = list(response[0]._json.keys())
    tweet_fields.remove('id')
    tweet_fields.remove('id_str')
    tweets = pd.DataFrame(columns=tweet_fields)
    tweets.index.name = "Tweet ID"
    
    for i in range(len(response)):
        tweet = response[i]

        # retrieving and formatting JSON of Tweet data for DataFrame
        tweet_data = tweet._json
        tweet_id = tweet_data['id']
        del tweet_data['id']
        del tweet_data['id_str']
        
        tweets.loc[tweet_id] = tweet_data
    
    return tweets

In [155]:
test30 = search_30("hello world", max_results=100)
test30

,created_at,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,...,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,matching_rules
Tweet ID,,,,,,,,,,,,,,,,,,,,,
1451277727647010816,Thu Oct 21 20:02:28 +0000 2021,"RT @MarkoSilberhand: Hello World,\n\nFull moon...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 898231845925646336, 'id_str': '89823184...",...,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1450907948063109132, 'id_str...",False,False,False,low,en,[{'tag': None}]
1451277668305956888,Thu Oct 21 20:02:14 +0000 2021,RT @TheMadTrooper: @Coinbase_NFT Hello! @Coinb...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,None,"{'id': 1361721650035580933, 'id_str': '1361721...",...,0,0,"{'hashtags': [{'text': 'MadTrooper', 'indices'...",NaN,False,False,NaN,low,en,[{'tag': None}]
1451277626149064712,Thu Oct 21 20:02:04 +0000 2021,"RT @MarkoSilberhand: Hello World,\n\nGreetings...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 898231845925646336, 'id_str': '89823184...",...,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1450820415304110080, 'id_str...",False,False,False,low,en,[{'tag': None}]
1451277606561607684,Thu Oct 21 20:01:59 +0000 2021,RT @SB19Official: 🎉 #SB19xATINAnnivMonth\n\nHe...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,None,"{'id': 1444279190581252098, 'id_str': '1444279...",...,0,0,"{'hashtags': [{'text': 'SB19xATINAnnivMonth', ...",NaN,False,False,NaN,low,en,[{'tag': None}]
1451277586328326144,Thu Oct 21 20:01:54 +0000 2021,"RT @MarkoSilberhand: Hello World,\n\nlast nigh...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 898231845925646336, 'id_str': '89823184...",...,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1450787246165090310, 'id_str...",False,False,False,low,en,[{'tag': None}]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451274007798497280,Thu Oct 21 19:47:41 +0000 2021,Hello World! Listen to this song! https://t.co...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 115451241, 'id_str': '115451241', 'name...",...,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,False,False,False,low,en,[{'tag': None}]
1451273895458258948,Thu Oct 21 19:47:14 +0000 2021,Hello World (1634845654),"<a href=""https://www.keysight.com/"" rel=""nofol...",False,None,None,None,None,None,"{'id': 930007300525510657, 'id_str': '93000730...",...,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,False,False,NaN,low,en,[{'tag': None}]
1451273840038793217,Thu Oct 21 19:47:01 +0000 2021,RT @TheMadTrooper: @Coinbase_NFT Hello! @Coinb...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,None,"{'id': 1400068616008081408, 'id_str': '1400068...",...,0,0,"{'hashtags': [{'text': 'MadTrooper', 'indices'...",NaN,False,False,NaN,low,en,[{'tag': None}]


In [150]:
test30["entities"].iloc[0]['hashtags'][0]['text']

'thecollective'

In [14]:
response_full = api1.search_full_archive(label="full", query="hello world lang:en", maxResults=20)

In [15]:
response_full[0]._json

{'created_at': 'Wed Oct 20 17:06:38 +0000 2021',
 'id': 1450871091174510593,
 'id_str': '1450871091174510593',
 'text': 'Hello World (1634749599)',
 'source': '<a href="https://www.keysight.com/" rel="nofollow">Nemo Outdoor</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 930007300525510657,
  'id_str': '930007300525510657',
  'name': 'Bench Mark',
  'screen_name': 'bnchmrk415',
  'location': None,
  'url': None,
  'description': None,
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 1,
  'friends_count': 0,
  'listed_count': 0,
  'favourites_count': 0,
  'statuses_count': 55931,
  'created_at': 'Mon Nov 13 09:39:53 +0000 2017',
  'utc_offset': None,
  'time_zone': None,
  'geo_enabled': False,
  'lang': None,
  'contributors_enabled': False,
  'is_translator': False,
  'profile_backgr

In [157]:
def search_full(query, start_date=None, end_date=None, max_results=20):
    # initializing API v1.1 instance
    api_1 = init_api_1()
    
    # parsing dates passed into function
    from dateutil import parser
    if start_date:
        start_date = parser.parse(start_date)
    if end_date:
        end_date = parser.parse(end_date)
    
    # retrieving Tweets from the past 30 days relevant to query
    response = api_1.search_full_archive(label="full",
                                         query=f"{query} lang:en",
                                         fromDate=start_date,
                                         toDate=end_date,
                                         maxResults=max_results)
    
    # creating DataFrame of Tweets
    import pandas as pd
    
    tweet_fields = list(response[0]._json.keys())
    tweet_fields.remove('id')
    tweet_fields.remove('id_str')
    tweets = pd.DataFrame(columns=tweet_fields)
    tweets.index.name = "Tweet ID"
    
    for i in range(len(response)):
        tweet = response[i]

        # retrieving and formatting JSON of Tweet data for DataFrame
        tweet_data = tweet._json
        tweet_id = tweet_data['id']
        del tweet_data['id']
        del tweet_data['id_str']
        
        tweets.loc[tweet_id] = tweet_data
    
    return tweets

In [158]:
test_full = search_full("hello world", max_results=100)
test_full

,created_at,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,...,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,possibly_sensitive,filter_level,lang,matching_rules
Tweet ID,,,,,,,,,,,,,,,,,,,,,
1451277794093248515,Thu Oct 21 20:02:44 +0000 2021,Promethean is a global leader in interactive d...,"<a href=""http://www.linkedin.com/"" rel=""nofoll...",True,None,None,None,None,None,"{'id': 1269354650, 'id_str': '1269354650', 'na...",...,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,False,low,en,[{'tag': None}]
1451277727647010816,Thu Oct 21 20:02:28 +0000 2021,"RT @MarkoSilberhand: Hello World,\n\nFull moon...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 898231845925646336, 'id_str': '89823184...",...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,False,low,en,[{'tag': None}]
1451277668305956888,Thu Oct 21 20:02:14 +0000 2021,RT @TheMadTrooper: @Coinbase_NFT Hello! @Coinb...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,None,"{'id': 1361721650035580933, 'id_str': '1361721...",...,0,0,0,"{'hashtags': [{'text': 'MadTrooper', 'indices'...",False,False,NaN,low,en,[{'tag': None}]
1451277626149064712,Thu Oct 21 20:02:04 +0000 2021,"RT @MarkoSilberhand: Hello World,\n\nGreetings...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 898231845925646336, 'id_str': '89823184...",...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,False,low,en,[{'tag': None}]
1451277606561607684,Thu Oct 21 20:01:59 +0000 2021,RT @SB19Official: 🎉 #SB19xATINAnnivMonth\n\nHe...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,None,"{'id': 1444279190581252098, 'id_str': '1444279...",...,0,0,0,"{'hashtags': [{'text': 'SB19xATINAnnivMonth', ...",False,False,NaN,low,en,[{'tag': None}]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451274202397425673,Thu Oct 21 19:48:27 +0000 2021,I haven't been on Twitter all day because my b...,"<a href=""http://twitter.com/download/android"" ...",True,None,None,None,None,None,"{'id': 48011948, 'id_str': '48011948', 'name':...",...,0,0,3,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,NaN,low,en,[{'tag': None}]
1451274007798497280,Thu Oct 21 19:47:41 +0000 2021,Hello World! Listen to this song! https://t.co...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,None,"{'id': 115451241, 'id_str': '115451241', 'name...",...,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,False,low,en,[{'tag': None}]
1451273895458258948,Thu Oct 21 19:47:14 +0000 2021,Hello World (1634845654),"<a href=""https://www.keysight.com/"" rel=""nofol...",False,None,None,None,None,None,"{'id': 930007300525510657, 'id_str': '93000730...",...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,NaN,low,en,[{'tag': None}]


## Stream

A Stream is an object that can filter and sample realtime Tweets.

In [11]:
# instantiating Stream object
stream = tweepy.Stream(consumer_key, consumer_secret, access_token, access_secret)
stream

In [ ]:
stream.sample(languages=["en"])